In [38]:
#coding:utf-8
import pandas as pd
import time

from sklearn.metrics import log_loss
# instance_id 样本编号
# item_id 广告商品编号
# item_category_list 广告商品的的类目列表 分割; item_property_list_0 item_property_list_1 item_property_list_2
# item_property_list 广告商品的属性列表 分割 1 2 3
# item_brand_id 广告商品的品牌编号
# item_city_id 广告商品的城市编号
# item_price_level 广告商品的价格等级
# item_sales_level 广告商品的销量等级
# item_collected_level 广告商品被收藏次数的等级
# item_pv_level 广告商品被展示次数的等级
# user_id 用户的编号
# 'user_gender_id', 用户的预测性别编号
# 'user_age_level', 用户的预测年龄等级
# 'user_occupation_id', 用户的预测职业编号
# 'user_star_level' 用户的星级编号
# context_id 上下文信息的编号
#  context_timestamp 广告商品的展示时间
# context_page_id 广告商品的展示页面编号
# predict_category_property
def time2cov(time_):
    '''
    时间是根据天数推移，所以日期为脱敏，但是时间本身不脱敏
    :param time_: 
    :return: 
    '''
    return time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time_))

def pre_process(data):
    '''
    :param data: 
    :return: 
    '''

    print('预处理')
    print('item_category_list_ing')
    for i in range(3):
        data['category_%d'%(i)] = data['item_category_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_category_list']

    print('item_property_list_ing')
    for i in range(3):
        data['property_%d'%(i)] = data['item_property_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_property_list']

    print('context_timestamp_ing')
    data['context_timestamp'] = data['context_timestamp'].apply(time2cov)

    print('time')
    data['context_timestamp_tmp'] = pd.to_datetime(data['context_timestamp'])
    data['week'] = data['context_timestamp_tmp'].dt.weekday
    data['hour'] = data['context_timestamp_tmp'].dt.hour
    data['minute'] = data['context_timestamp_tmp'].dt.minute
    data['halfhour'] = (data['hour']+1)*3+(data['minute']/20).astype('int')
    del data['context_timestamp_tmp']
    del data['minute']
    del data['week']
    del data['hour']


#     print('predict_category_property_ing_0')
#     for i in range(3):
#         data['predict_category_%d'%(i)] = data['predict_category_property'].apply(
#             lambda x:str(x.split(";")[i]).split(":")[0] if len(x.split(";")) > i else " "
#         )
#     for i in range(3):
#         temp = data[['user_occupation_id','property_%d'%(i),'instance_id']].groupby(['user_occupation_id','property_%d'%(i)]).aggregate('count')
#         temp1 = temp.reset_index()
#         temp2 = temp.groupby('user_occupation_id').sum().reset_index()
#         temp3 = pd.merge(temp1,temp2,on=['user_occupation_id'],how='left')
#         temp4 = temp3[['instance_id_x']].rename(columns=({'instance_id_x':'sum'}))/temp3[['instance_id_y']].rename(columns=({'instance_id_y':'sum'}))
#         temp3['property_%dpv'%(i)] = temp4
#         temp3.drop('instance_id_x', axis=1, inplace=True)
#         temp3.drop('instance_id_y', axis=1, inplace=True)
#         data = pd.merge(data,temp3,on=['user_occupation_id','property_%d'%(i)],how='left')
#     for i in range(1,3):
#         temp = data[['user_occupation_id','category_%d'%(i),'instance_id']].groupby(['user_occupation_id','category_%d'%(i)]).aggregate('count')
#         temp1 = temp.reset_index()
#         temp2 = temp.groupby('user_occupation_id').sum().reset_index()
#         temp3 = pd.merge(temp1,temp2,on=['user_occupation_id'],how='left')
#         temp4 = temp3[['instance_id_x']].rename(columns=({'instance_id_x':'sum'}))/temp3[['instance_id_y']].rename(columns=({'instance_id_y':'sum'}))
#         temp3['category_%dpv'%(i)] = temp4
#         temp3.drop('instance_id_x', axis=1, inplace=True)
#         temp3.drop('instance_id_y', axis=1, inplace=True)
#         data = pd.merge(data,temp3,on=['user_occupation_id','category_%d'%(i)],how='left')
#     for i in range(3):
#         temp = data[['user_gender_id','property_%d'%(i),'instance_id']].groupby(['user_gender_id','property_%d'%(i)]).aggregate('count')
#         temp1 = temp.reset_index()
#         temp2 = temp.groupby('user_gender_id').sum().reset_index()
#         temp3 = pd.merge(temp1,temp2,on=['user_gender_id'],how='left')
#         temp4 = temp3[['instance_id_x']].rename(columns=({'instance_id_x':'sum'}))/temp3[['instance_id_y']].rename(columns=({'instance_id_y':'sum'}))
#         temp3['property_%dpv'%(i)] = temp4
#         temp3.drop('instance_id_x', axis=1, inplace=True)
#         temp3.drop('instance_id_y', axis=1, inplace=True)
#         data = pd.merge(data,temp3,on=['user_gender_id','property_%d'%(i)],how='left')
#     for i in range(1,3):
#         temp = data[['user_gender_id','category_%d'%(i),'instance_id']].groupby(['user_gender_id','category_%d'%(i)]).aggregate('count')
#         temp1 = temp.reset_index()
#         temp2 = temp.groupby('user_gender_id').sum().reset_index()
#         temp3 = pd.merge(temp1,temp2,on=['user_gender_id'],how='left')
#         temp4 = temp3[['instance_id_x']].rename(columns=({'instance_id_x':'sum'}))/temp3[['instance_id_y']].rename(columns=({'instance_id_y':'sum'}))
#         temp3['category_%dpv'%(i)] = temp4
#         temp3.drop('instance_id_x', axis=1, inplace=True)
#         temp3.drop('instance_id_y', axis=1, inplace=True)
#         data = pd.merge(data,temp3,on=['user_gender_id','category_%d'%(i)],how='left')
#     temp = data[['user_age_level','item_price_level','instance_id']].groupby(['user_age_level','item_price_level']).aggregate('count')
#     temp1 = temp.reset_index()
#     temp2 = temp.groupby('user_age_level').sum().reset_index()
#     temp3 = pd.merge(temp1,temp2,on=['user_age_level'],how='left')
#     temp4 = temp3[['instance_id_x']].rename(columns=({'instance_id_x':'sum'}))/temp3[['instance_id_y']].rename(columns=({'instance_id_y':'sum'}))
#     temp3['age_pv'] = temp4
#     temp3.drop('instance_id_x', axis=1, inplace=True)
#     temp3.drop('instance_id_y', axis=1, inplace=True)
#     data = pd.merge(data,temp3,on=['user_age_level','user_age_level'],how='left')
#     del(temp3)
#     del(temp2)
#     del(temp1)
#     del(temp4)
#     del(temp)
#     print('predict_category_property_ing_1')
#     for i in range(3):
#         data['predict_property_%d'%(i)] = data['predict_category_property'].apply(
#             lambda x:str(x.split(";")[i]).split(":")[1] if len(x.split(";")) > i else " "
#         )
    
#         for j in range(3):
#             data['predict_property_%d_%d' % (i,j)] = data['predict_property_%d'%(i)].apply(
#                 lambda x: x.split(",")[j] if len(x.split(",")) > j else -1
#             )

    del data['predict_category_property']
    # del data['predict_property_1']
    # del data['predict_property_2']
    del data['shop_score_service']
    del data['shop_score_description']
    del data['shop_review_positive_rate']
    del data['shop_score_delivery']
    return data

In [39]:
print('train')
train = pd.read_csv('train.csv',index_col=False)
dataa = pd.read_csv('train_all.csv',index_col=False) 
train = pd.merge(train,dataa,on=['instance_id'],how='left')
train = pre_process(train)



train
预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time


In [40]:
all_data = train.copy()


print('all_shape',train.shape)
print(train['context_timestamp'].max())
val = train[train['context_timestamp']>'2018-09-22 23:59:59']

# 空去 22 号

train = train[train['context_timestamp']<='2018-09-22 23:59:59']

# train = train[train['context_timestamp']>'2018-09-19 23:59:59']
print(train.shape)
print(val.shape)


all_shape (478087, 41)
2018-09-24 23:59:47
(357066, 41)
(121021, 41)


In [41]:

print('test')
test_a = pd.read_csv('test.csv',index_col=False)
data2a = pd.read_csv('test_all.csv',index_col=False) 
test_a = pd.merge(test_a,data2a,on=['instance_id'],how='left')
print(test_a.shape)
test_a = pre_process(test_a)
del train['item_city_id']
del val['item_city_id']
del test_a['item_city_id']
del train['shop_id']
del val['shop_id']
del test_a['shop_id']
# 0.83108
# del train[['user_id', 'category_1', 'property_2', 'property_0']]
# del val[['user_id', 'category_1', 'property_2', 'property_0']]
# del test_a[['user_id', 'category_1', 'property_2', 'property_0']]
# 这里是增加的内容


test
(18371, 51)
预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time


In [42]:

import datetime
def get_count_feat(all_data,data,long=3):
    end_time = data['context_timestamp'].min()
    begin_time = pd.to_datetime(end_time) - datetime.timedelta(days=long)
    all_data['context_timestamp'] = pd.to_datetime(all_data['context_timestamp'])
    all_data = all_data[
        (all_data['context_timestamp']<end_time)&(all_data['context_timestamp']>=begin_time)
                    ]
    print(end_time)
    print(begin_time)
    print(all_data['context_timestamp'].max()-all_data['context_timestamp'].min())
    item_count = all_data.groupby(['item_id'],as_index=False).size().reset_index()
    item_count.rename(columns={0:'item_count'},inplace=True)

    user_count = all_data.groupby(['user_id'], as_index=False).size().reset_index()
    user_count.rename(columns={0: 'user_count'}, inplace=True)
    return user_count,item_count




In [43]:

train_user_count,train_item_count = get_count_feat(all_data,train,2)

test_user_count,test_item_count = get_count_feat(all_data,test_a,2)

val_user_count,val_item_count = get_count_feat(all_data,val,2)
# print(train.columns)
train = pd.merge(train,train_user_count,on=['user_id'],how='left')
train = pd.merge(train,train_item_count,on=['item_id'],how='left')
train = train.fillna(-1)
val = pd.merge(val,val_user_count,on=['user_id'],how='left')
val = pd.merge(val,val_item_count,on=['item_id'],how='left')
val = val.fillna(-1)
test_a = pd.merge(test_a,test_user_count,on=['user_id'],how='left')
test_a = pd.merge(test_a,test_item_count,on=['item_id'],how='left')
test_a = test_a.fillna(-1)

# 删掉上面增加的用户点击量与商品点击量
# del train['user_count']
# del train['item_count']
# del val['user_count']
# del val['item_count']
# del test_a['user_count']
# del test_a['item_count']

# user_item_count = pd.read_csv


y_train = train.pop('is_trade')
train_index = train.pop('instance_id')

y_val = val.pop('is_trade')
val_index = val.pop('instance_id')
test_index = test_a.pop('instance_id')

print(test_a.shape)
del train['context_timestamp']
del val['context_timestamp']
del test_a['context_timestamp']
del all_data


2018-09-18 00:00:01
2018-09-16 00:00:01
NaT
2018-09-25 00:00:02
2018-09-23 00:00:02
1 days 23:59:44
2018-09-23 00:00:03
2018-09-21 00:00:03
1 days 23:59:43
(18371, 50)


In [44]:

print('baseline ing ... ...')
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from scipy import sparse
from sklearn.linear_model import LogisticRegression
# print(test_a.columns)

enc = OneHotEncoder()
lb = LabelEncoder()
# feat_set = list(test_a.columns)

del train['user_id']
del test_a['user_id']
del val['user_id']
# 8288

print(train.columns)

# 自己使用onehot编码
train['x_train'] = 1
test_a['x_test'] = 1
val['x_val'] = 1

all_data = pd.concat([train,test_a,val])
all_data = pd.get_dummies(all_data)
print(all_data.shape)

X_train = all_data.loc[all_data['x_train'].notnull()]
X_test = all_data.loc[all_data['x_test'].notnull()]
X_val = all_data.loc[all_data['x_val'].notnull()]

del X_train['x_train']
del X_train['x_test']
del X_train['x_val']
del X_test['x_train']
del X_test['x_test']
del X_test['x_val']
del X_val['x_train']
del X_val['x_test']
del X_val['x_val']


baseline ing ... ...
Index(['item_id', 'item_brand_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_page_id', 'shop_review_num_level', 'shop_star_level',
       'item_category_list_num', 'item_property_list_num', 'hour_num',
       'item_click', 'shop_score_service_num', 'shop_score_description_num',
       'shop_review_positive_rate_num', 'shop_score_delivery_num',
       'shop_click', 'category_1_pvnum', 'property_0_pvnum',
       'property_1_pvnum', 'property_2_pvnum', 'item_id_pvnum', 'category_0',
       'category_1', 'category_2', 'property_0', 'property_1', 'property_2',
       'halfhour', 'user_count', 'item_count'],
      dtype='object')
(496458, 614)


In [45]:
# LogisticRegression?

In [46]:

# lr = LogisticRegression(C=5.0,penalty='l1')

# lr.fit(X_train, y_train)


In [47]:

# proba_val = lr.predict_proba(X_val)[:,1]
# proba_sub = lr.predict_proba(X_test)[:,1]
# print(log_loss(y_train,lr.predict_proba(X_train)[:,1]))
# print(log_loss(y_val,proba_val))
# print()

In [48]:

import lightgbm as lgb
gbm = lgb.LGBMRegressor(objective='binary',
                        num_leaves=50,
                        learning_rate=0.01,
                        n_estimators=2000,
                        colsample_bytree = 0.65,
                        subsample = 0.7,
                        reg_alpha = 1
                       )
gbm.fit(X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='binary_logloss',
        early_stopping_rounds=150)



[1]	valid_0's binary_logloss: 0.6839
Training until validation scores don't improve for 150 rounds.
[2]	valid_0's binary_logloss: 0.67484
[3]	valid_0's binary_logloss: 0.665963
[4]	valid_0's binary_logloss: 0.657248
[5]	valid_0's binary_logloss: 0.648701
[6]	valid_0's binary_logloss: 0.640313
[7]	valid_0's binary_logloss: 0.632093
[8]	valid_0's binary_logloss: 0.624024
[9]	valid_0's binary_logloss: 0.616099
[10]	valid_0's binary_logloss: 0.608332
[11]	valid_0's binary_logloss: 0.600702
[12]	valid_0's binary_logloss: 0.593204
[13]	valid_0's binary_logloss: 0.585843
[14]	valid_0's binary_logloss: 0.57862
[15]	valid_0's binary_logloss: 0.571523
[16]	valid_0's binary_logloss: 0.564558
[17]	valid_0's binary_logloss: 0.557701
[18]	valid_0's binary_logloss: 0.550972
[19]	valid_0's binary_logloss: 0.54436
[20]	valid_0's binary_logloss: 0.537867
[21]	valid_0's binary_logloss: 0.531485
[22]	valid_0's binary_logloss: 0.525197
[23]	valid_0's binary_logloss: 0.519021
[24]	valid_0's binary_logloss: 

[204]	valid_0's binary_logloss: 0.125136
[205]	valid_0's binary_logloss: 0.124593
[206]	valid_0's binary_logloss: 0.124056
[207]	valid_0's binary_logloss: 0.123524
[208]	valid_0's binary_logloss: 0.123003
[209]	valid_0's binary_logloss: 0.122486
[210]	valid_0's binary_logloss: 0.121974
[211]	valid_0's binary_logloss: 0.12147
[212]	valid_0's binary_logloss: 0.120973
[213]	valid_0's binary_logloss: 0.120482
[214]	valid_0's binary_logloss: 0.119995
[215]	valid_0's binary_logloss: 0.119513
[216]	valid_0's binary_logloss: 0.11904
[217]	valid_0's binary_logloss: 0.118573
[218]	valid_0's binary_logloss: 0.11811
[219]	valid_0's binary_logloss: 0.117654
[220]	valid_0's binary_logloss: 0.117203
[221]	valid_0's binary_logloss: 0.116758
[222]	valid_0's binary_logloss: 0.116317
[223]	valid_0's binary_logloss: 0.115882
[224]	valid_0's binary_logloss: 0.115453
[225]	valid_0's binary_logloss: 0.11503
[226]	valid_0's binary_logloss: 0.114613
[227]	valid_0's binary_logloss: 0.1142
[228]	valid_0's binary

[403]	valid_0's binary_logloss: 0.0856448
[404]	valid_0's binary_logloss: 0.0856045
[405]	valid_0's binary_logloss: 0.0855651
[406]	valid_0's binary_logloss: 0.0855239
[407]	valid_0's binary_logloss: 0.0854837
[408]	valid_0's binary_logloss: 0.0854461
[409]	valid_0's binary_logloss: 0.0854082
[410]	valid_0's binary_logloss: 0.0853721
[411]	valid_0's binary_logloss: 0.0853359
[412]	valid_0's binary_logloss: 0.0852978
[413]	valid_0's binary_logloss: 0.0852594
[414]	valid_0's binary_logloss: 0.0852242
[415]	valid_0's binary_logloss: 0.085188
[416]	valid_0's binary_logloss: 0.085151
[417]	valid_0's binary_logloss: 0.0851168
[418]	valid_0's binary_logloss: 0.0850806
[419]	valid_0's binary_logloss: 0.0850469
[420]	valid_0's binary_logloss: 0.085015
[421]	valid_0's binary_logloss: 0.084985
[422]	valid_0's binary_logloss: 0.0849525
[423]	valid_0's binary_logloss: 0.0849202
[424]	valid_0's binary_logloss: 0.0848841
[425]	valid_0's binary_logloss: 0.0848517
[426]	valid_0's binary_logloss: 0.0848

[600]	valid_0's binary_logloss: 0.0825629
[601]	valid_0's binary_logloss: 0.0825606
[602]	valid_0's binary_logloss: 0.0825552
[603]	valid_0's binary_logloss: 0.0825502
[604]	valid_0's binary_logloss: 0.0825475
[605]	valid_0's binary_logloss: 0.0825436
[606]	valid_0's binary_logloss: 0.082539
[607]	valid_0's binary_logloss: 0.0825347
[608]	valid_0's binary_logloss: 0.0825301
[609]	valid_0's binary_logloss: 0.082524
[610]	valid_0's binary_logloss: 0.0825208
[611]	valid_0's binary_logloss: 0.0825156
[612]	valid_0's binary_logloss: 0.0825133
[613]	valid_0's binary_logloss: 0.0825074
[614]	valid_0's binary_logloss: 0.0825029
[615]	valid_0's binary_logloss: 0.0825011
[616]	valid_0's binary_logloss: 0.0824962
[617]	valid_0's binary_logloss: 0.0824928
[618]	valid_0's binary_logloss: 0.0824885
[619]	valid_0's binary_logloss: 0.0824826
[620]	valid_0's binary_logloss: 0.0824792
[621]	valid_0's binary_logloss: 0.0824764
[622]	valid_0's binary_logloss: 0.0824741
[623]	valid_0's binary_logloss: 0.08

[798]	valid_0's binary_logloss: 0.0821507
[799]	valid_0's binary_logloss: 0.0821484
[800]	valid_0's binary_logloss: 0.0821471
[801]	valid_0's binary_logloss: 0.0821455
[802]	valid_0's binary_logloss: 0.0821445
[803]	valid_0's binary_logloss: 0.0821438
[804]	valid_0's binary_logloss: 0.0821431
[805]	valid_0's binary_logloss: 0.0821428
[806]	valid_0's binary_logloss: 0.0821402
[807]	valid_0's binary_logloss: 0.0821412
[808]	valid_0's binary_logloss: 0.0821422
[809]	valid_0's binary_logloss: 0.0821425
[810]	valid_0's binary_logloss: 0.0821436
[811]	valid_0's binary_logloss: 0.0821429
[812]	valid_0's binary_logloss: 0.0821433
[813]	valid_0's binary_logloss: 0.0821412
[814]	valid_0's binary_logloss: 0.0821412
[815]	valid_0's binary_logloss: 0.0821398
[816]	valid_0's binary_logloss: 0.0821397
[817]	valid_0's binary_logloss: 0.0821384
[818]	valid_0's binary_logloss: 0.0821378
[819]	valid_0's binary_logloss: 0.0821374
[820]	valid_0's binary_logloss: 0.0821367
[821]	valid_0's binary_logloss: 0.

[999]	valid_0's binary_logloss: 0.082067
[1000]	valid_0's binary_logloss: 0.0820676
[1001]	valid_0's binary_logloss: 0.082066
[1002]	valid_0's binary_logloss: 0.0820664
[1003]	valid_0's binary_logloss: 0.0820673
[1004]	valid_0's binary_logloss: 0.0820665
[1005]	valid_0's binary_logloss: 0.0820649
[1006]	valid_0's binary_logloss: 0.0820651
[1007]	valid_0's binary_logloss: 0.0820671
[1008]	valid_0's binary_logloss: 0.0820674
[1009]	valid_0's binary_logloss: 0.082068
[1010]	valid_0's binary_logloss: 0.0820669
[1011]	valid_0's binary_logloss: 0.0820651
[1012]	valid_0's binary_logloss: 0.0820647
[1013]	valid_0's binary_logloss: 0.0820633
[1014]	valid_0's binary_logloss: 0.082063
[1015]	valid_0's binary_logloss: 0.0820633
[1016]	valid_0's binary_logloss: 0.0820626
[1017]	valid_0's binary_logloss: 0.082062
[1018]	valid_0's binary_logloss: 0.0820631
[1019]	valid_0's binary_logloss: 0.0820614
[1020]	valid_0's binary_logloss: 0.0820615
[1021]	valid_0's binary_logloss: 0.082061
[1022]	valid_0's b

[1194]	valid_0's binary_logloss: 0.0820543
[1195]	valid_0's binary_logloss: 0.0820532
[1196]	valid_0's binary_logloss: 0.0820521
[1197]	valid_0's binary_logloss: 0.0820524
[1198]	valid_0's binary_logloss: 0.0820488
[1199]	valid_0's binary_logloss: 0.0820474
[1200]	valid_0's binary_logloss: 0.0820464
[1201]	valid_0's binary_logloss: 0.0820464
[1202]	valid_0's binary_logloss: 0.0820475
[1203]	valid_0's binary_logloss: 0.0820479
[1204]	valid_0's binary_logloss: 0.0820482
[1205]	valid_0's binary_logloss: 0.0820487
[1206]	valid_0's binary_logloss: 0.0820495
[1207]	valid_0's binary_logloss: 0.0820501
[1208]	valid_0's binary_logloss: 0.0820502
[1209]	valid_0's binary_logloss: 0.0820501
[1210]	valid_0's binary_logloss: 0.0820499
[1211]	valid_0's binary_logloss: 0.082051
[1212]	valid_0's binary_logloss: 0.0820499
[1213]	valid_0's binary_logloss: 0.0820494
[1214]	valid_0's binary_logloss: 0.0820493
[1215]	valid_0's binary_logloss: 0.0820498
[1216]	valid_0's binary_logloss: 0.0820484
[1217]	valid

[1388]	valid_0's binary_logloss: 0.0820578
[1389]	valid_0's binary_logloss: 0.0820571
[1390]	valid_0's binary_logloss: 0.082058
[1391]	valid_0's binary_logloss: 0.0820574
[1392]	valid_0's binary_logloss: 0.0820578
[1393]	valid_0's binary_logloss: 0.0820571
[1394]	valid_0's binary_logloss: 0.0820574
[1395]	valid_0's binary_logloss: 0.0820567
[1396]	valid_0's binary_logloss: 0.0820559
[1397]	valid_0's binary_logloss: 0.0820547
[1398]	valid_0's binary_logloss: 0.082054
[1399]	valid_0's binary_logloss: 0.0820532
[1400]	valid_0's binary_logloss: 0.0820538
[1401]	valid_0's binary_logloss: 0.0820536
[1402]	valid_0's binary_logloss: 0.0820543
[1403]	valid_0's binary_logloss: 0.0820538
[1404]	valid_0's binary_logloss: 0.0820552
[1405]	valid_0's binary_logloss: 0.0820547
[1406]	valid_0's binary_logloss: 0.0820538
[1407]	valid_0's binary_logloss: 0.0820542
[1408]	valid_0's binary_logloss: 0.082055
[1409]	valid_0's binary_logloss: 0.0820551
[1410]	valid_0's binary_logloss: 0.0820552
[1411]	valid_0

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.65,
       learning_rate=0.01, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=2000,
       n_jobs=-1, num_leaves=50, objective='binary', random_state=None,
       reg_alpha=1, reg_lambda=0.0, silent=True, subsample=0.7,
       subsample_for_bin=200000, subsample_freq=1)

In [49]:
print('Start predicting...')
# predict
y_pred_1 = gbm.predict(X_val, num_iteration=gbm.best_iteration_ )
y_sub_1 = gbm.predict(X_test, num_iteration=gbm.best_iteration_ )
print(log_loss(y_train,gbm.predict(X_train)))
print(log_loss(y_val,y_pred_1))

# print(log_loss(y_val,(y_pred_1 + proba_val)/2))#0.0816995451384

Start predicting...
0.0798704929739
0.0820377010162


In [50]:

# import numpy as np
# bes = 1
# for i in np.arange(0,1,0.01):
#     if(log_loss(y_val,(i * y_pred_1 + (1-i) * proba_val)) < bes):
#         bes = log_loss(y_val,(y_pred_1 + proba_val)/2)
# print(bes)#0.083131236937

# xx_analyse = pd.DataFrame()
# xx_analyse['ture'] = list(y_val)
# xx_analyse['pre'] = list(proba_val)
# xx_analyse['pre_1'] = list(y_pred_1)
# xx_analyse.to_csv('temp.csv',index=False)
#
# sub = pd.DataFrame()
# sub['instance_id'] = list(test_index)
# sub['predicted_score'] = list(y_sub_1)
# sub.to_csv('feather/20180327.txt',sep=" ",index=False)

In [51]:
sub = pd.DataFrame()
sub['instance_id'] = list(test_index)
sub['predicted_score'] = list(y_sub_1)
sub.to_csv('20180410.txt',sep=" ",index=False)